# Generate snippet images

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import utils.GenSnippetsLib as gsl
from PIL import Image
import os
import matplotlib
import json
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
snippets: list[str] = []

with open(f'./data/my_snippets.csv', mode="r") as f:
	for line in f:
		line_split = line.split(';')
		if line_split[0] == 'Title':
			continue
		
		file_name = line_split[0]
		snippets.append(file_name)
		
		# file_content = line_split[1].replace('\\n', '\n').replace('\\t', '    ')

		# with open(f'./data/source_code/{file_name}.py', 'w') as file:
		# 	file.write(file_content)


## Import Eyetracking Data

In [ ]:
# here set the snippets to generate images in the data folder from
# snippets = ['unrolledSortMT']


# image size for fixed image sizes
IMAGE_SIZE_FIXED = True
IMAGE_SIZE = (2560, 1440)# width, height

In [ ]:
image_generator_template = {
    'font-normal': 'JetBrainsMonoNL-Regular.ttf',
    'font-bold': 'JetBrainsMonoNL-Bold.ttf',
    'font-italic': 'JetBrainsMonoNL-Italic.ttf',
    'font-bold-italic': 'JetBrainsMonoNL-BoldItalic.ttf',
    'font-size': 50,
    'width-margin': 5,  # buffer in width between image and code border
    'height-margin': 5,  # buffer in height between image and code border
    'spacing': 40,  # distance between code lines
    'background-color': [255, 255, 255],  # White
    # 'style': 'vs',
	'style': 'NoStyle',
    'styles': [
    #     'abap',  # 3 no coloring constant numbers
    #     'algol',  # 1 for dark background or no coloring
    #     'algol_nu',  # 1 for dark background or no coloring
    #     'arduino',  # 2 too many colors and intensity variations
    #     'autumn',  # first variable not bold
    #     'borland',  # 3 no coloring constant numbers
    #     'bw',  # 1 for dark background or no coloring
    #     'colorful',  # 2 too many colors and intensity variations
    #     'default',  # 2 too many colors and intensity variations, including red
    #     'dracula',  # 1 for dark background or no coloring
    #     'emacs',  # 2 colors too contrasting & bright
    #     'friendly',  # 2 too many colors and intensity variations, including red
    #     'friendly_grayscale',  # 1 for dark background or no coloring
    #     'fruity',  # 1 for dark background or no coloring
    #     'github-dark',  # 1 for dark background or no coloring
    #     'gruvbox-dark',  # 1 for dark background or no coloring
    #     'gruvbox-light',  # 2 too many colors and intensity variations, including red
    #     'igor',  # 4 only two colors, vs looks better with colors and bold style
    #     'inkpot',  # 1 for dark background or no coloring
    #     'lilypond',  # 2 too little colors (only numbers)
    #     'lovelace',  # 2 italic
    #     'manni',  # 2 orange
    #     'material',  # 1 for dark background or no coloring
    #     'monokai',  # 1 for dark background or no coloring
    #     'murphy',  # 3 no coloring constant numbers
    #     'native',  # 1 for dark background or no coloring
    #     'nord',  # 1 for dark background or no coloring
    #     'nord-darker',  # 1 for dark background or no coloring
    #     'one-dark',  # 1 for dark background or no coloring
    #     'paraiso-dark',  # 1 for dark background or no coloring
    #     'paraiso-light',  # 2 too many colors and intensity variations, including orange
    #     'pastie',  # 2 too many colors and intensity variations
    #     'perldoc',  # 2 too many colors and intensity variations
    #     'rainbow_dash',  # 2 too many colors and intensity variations
    #     'rrt',  # 1 for dark background or no coloring
    #     'sas',  # 2 first variable not bold
    #     'solarized-dark',  # 1 for dark background or no coloring
    #     'solarized-light',  # 2 too many colors and intensity variations, including orange
    #     'staroffice',  # 2 too many colors and intensity variations, including red
    #     'stata',  # 3 no coloring constant numbers
    #     'stata-dark',  # 1 for dark background or no coloring
    #     'tango',  # 2 too many colors and intensity variations, including orange
    #     'trac',  # 3 no coloring constant numbers
    #     'vim',  # 1 for dark background or no coloring
    #     'vs', # CHOICE: 4 three colors (types, keywords, rest), looks better with colors and bold style than igor
    #     'xcode',  # 2 pink
    #     'zenburn'  # 1 for dark background or no coloring
    ]
	# base16-grayscale-light
}

In [ ]:
def save_aoi_list(aoi_list, file_name, story_order, execution_order):
    aoi_data: dict = {}
    aoi_data['AOIs'] = []
    for ele in aoi_list:
        aoi_data['AOIs'].append({'name': ele[0], 'x_left': ele[1], 'y_bottom': ele[2], 'x_right': ele[3], 'y_top': ele[4], 'color': ele[5]})
    
    # aoi_dataframe['AOIs'] = pd.DataFrame(aoi_list, columns=['index', 'name', 'x_left', 'y_top', 'x_right', 'y_bottom', 'color'])
    # aoi_dataframe['AOIs'].drop(columns=['index'], inplace=True, axis=1)
    # aoi_dataframe['AOIs']['name'] = aoi_dataframe['AOIs']['name'].astype(str)
    # aoi_dataframe['AOIs']['x_left'] = aoi_dataframe['AOIs']['x_left'].astype(int)
    # aoi_dataframe['AOIs']['y_top'] = aoi_dataframe['AOIs']['y_top'].astype(int)
    # aoi_dataframe['AOIs']['x_right'] = aoi_dataframe['AOIs']['x_right'].astype(int)
    # aoi_dataframe['AOIs']['y_bottom'] = aoi_dataframe['AOIs']['y_bottom'].astype(int)

    aoi_data['story_order'] = story_order
    aoi_data['execution_order'] = execution_order

    with open(f'./data/aoi_lists/{file_name}.json', 'w') as file:
        json.dump(aoi_data, file)

# AOI based analysis

In [ ]:
# Get AOI based AOIs
for snippet in tqdm(snippets):
    # with open(f'./data/source_code/{snippet}.py', 'r', encoding='utf-8') as f:
    #     source_code = f.read()

    # # create image generator based on snippet code and template above
    # template = image_generator_template.copy()
    # template['source-code'] = source_code.splitlines()
    # os.makedirs("./data/image_generator/", exist_ok=True)
    image_generator_path = f'./data/image_generator/{snippet}.json'
    # with open(image_generator_path, 'w', encoding='utf-8') as f:
    #     json.dump(template, f, indent=4, separators=(',', ': '))
    template = json.load(open(image_generator_path, 'r', encoding='utf-8'))

    # create image from image generator (json file with all parameters)
    try:
        image, aoi_list = gsl.create_image(image_generator_path, font_path="/../fonts/ttf/")
    except:
        print(f"{snippet} failed")
        image, aoi_list = gsl.create_image(image_generator_path, font_path="/../fonts/ttf/", logging=True)
        continue

    prev_image_size = (0, 0)
    if IMAGE_SIZE_FIXED:
        if IMAGE_SIZE[0] < image.size[0] or IMAGE_SIZE[1] < image.size[1]:
            print('error: image larger than fixed image size')
        else:
            # set image to fixed size
            image_fixed= Image.new('RGB', IMAGE_SIZE, tuple(template['background-color']))
            image_fixed.paste(image, ((IMAGE_SIZE[0] // 2) - (image.width // 2), (IMAGE_SIZE[1] // 2) - (image.height // 2))) # this was (0, 0) beforehand
            prev_image_size = image.size
            image = image_fixed
        
    # create aoi boxes around specifically marked areas
    aoi_list = gsl.generate_AOI_boxes(image,  aoi_list, prev_image_size)
    try:
        save_aoi_list(aoi_list, snippet, template['story_order'], template['execution_order'])
    except:
        print(f"{snippet} failed")
        save_aoi_list(aoi_list, snippet, [], [])

    # store image
    os.makedirs("./data/images/", exist_ok=True)
    image.save(f"./data/images/{snippet}.png")
